In [1]:
using Cropbox

WebIO._IJuliaInit()

In [30]:
using DataFrames
using Dates
import YAML
import Pkg
using Random
using Plots
#using TimeZones

In [3]:
using Garlic

In [16]:
using DataFrames

# 시뮬레이션 결과를 데이터프레임으로 변환하는 함수
function simulate_to_dataframe(result)
    # 결과를 데이터프레임으로 변환
    df = DataFrame(result)
    return df
end


simulate_to_dataframe (generic function with 1 method)

In [9]:
pheno_config = @config(
    :Calendar  => :init          => ZonedDateTime(2010, 10, 1, tz"Asia/Seoul"),
    :Weather   => :store         => Garlic.loadwea(Garlic.datapath("Korea/185_2010.wea"), tz"Asia/Seoul"),
    :Phenology => :planting_date => ZonedDateTime(2010, 11, 1, tz"Asia/Seoul"),
)
    

init,=,"ZonedDateTime(2010, 10, 1, tz""Asia/Seoul"")"
store,=,8760×10 DataFrame…
planting_date,=,"ZonedDateTime(2010, 11, 1, tz""Asia/Seoul"")"


In [4]:
config = Garlic.Examples.RCP.ND_RICCA_2014_field

optimal_temperature,=,22.28
ceiling_temperature,=,34.23
critical_photoperiod,=,12
maximum_leaf_initiation_rate,=,0.1003
storage_temperature,=,8
maximum_phyllochron_asymptote,=,0.4421
leaves_generic,=,10
initial_leaves_at_harvest,=,6
scape_appearance_threshold,=,3.0
planting_date,=,"ZonedDateTime(2014, 10, 8, tz""Asia/Seoul"")"
scape_removal_date,=,nothing


In [4]:
config = Garlic.Examples.RCP.ND_RICCA_2014_field

optimal_temperature,=,22.28
ceiling_temperature,=,34.23
critical_photoperiod,=,12
maximum_leaf_initiation_rate,=,0.1003
storage_temperature,=,8
maximum_phyllochron_asymptote,=,0.4421
leaves_generic,=,10
initial_leaves_at_harvest,=,6
scape_appearance_threshold,=,3.0
planting_date,=,"ZonedDateTime(2014, 10, 8, tz""Asia/Seoul"")"
scape_removal_date,=,nothing


In [37]:
# 시뮬레이션 수 설정
num_simulations = 1

# 결과를 저장할 리스트
leaf_results = []
params = []  # 매개변수 값을 저장할 리스트

# 시뮬레이션 실행
for i in 1:num_simulations
    # Leaf 매개변수에 대해 랜덤 값 생성
    max_elongation_rate = rand(3.0:0.1:7.0)
    min_length_longest_leaf = rand(70.0:1.0:110.0)
    stay_green = rand(1.0:0.1:6.0)
    
    # 현재 시뮬레이션의 Leaf 구성
    leaf_config = deepcopy(config)
    leaf_config[:Leaf][:maximum_elongation_rate] = max_elongation_rate
    leaf_config[:Leaf][:minimum_length_of_longest_leaf] = min_length_longest_leaf
    leaf_config[:Leaf][:stay_green] = stay_green

    # 시뮬레이션 실행
    result = simulate(Garlic.Model;
        config = leaf_config,
        index = :time,
        target = [],
        stop = "calendar.count",
        snap = 1u"d"
    )

    # 결과를 데이터프레임으로 변환
    result_df = DataFrame(result)
    push!(leaf_results, result_df)

    # 매개변수 값 기록
    push!(params, (max_elongation_rate, min_length_longest_leaf, stay_green))
end

# 데이터프레임 열 이름 출력
println("Column names of the result DataFrame: ")
for result_df in leaf_results
    println(names(result_df))  # 열 이름을 출력하여 확인합니다.
end

# 시뮬레이션 결과를 시각화
function plot_simulation_results(results, params)
    for (i, result_df) in enumerate(results)
        # 데이터프레임에서 시간 열 추출
        time = result_df[:, :time]  # 시간 열
        
        # 매개변수 값 추출
        max_elongation_rate, min_length_longest_leaf, stay_green = params[i]

        # 플롯 생성
        p = Plots.plot(title="Simulation Results (Simulation $i)", xlabel="Time", ylabel="Value")
        
        Plots.plot!(p, time, label="Time vs. Values")
        scatter!(p, time, label="Time Points", marker=:circle)

        # 매개변수 값을 그래프에 주석으로 추가
        annotate!(p, [(mean(time), max_elongation_rate, text("Max Elongation Rate: $max_elongation_rate", :left)),
                      (mean(time), min_length_longest_leaf, text("Min Length of Longest Leaf: $min_length_longest_leaf", :left)),
                      (mean(time), stay_green, text("Stay Green: $stay_green", :left))])

        # 그래프 표시
        display(p)
    end
end

plot_simulation_results(leaf_results, params)


Progress: 100%[===========================] Time: 0:00:04 ( 0.65 ms/it)


Column names of the result DataFrame: 
["time"]


LoadError: Cannot convert ZonedDateTime to series data for plotting

In [ ]:
visualize(leaf_sim, :time, (:max_elongation_rate, :min_length_longest_leaf, :stay_green);, kind=:line)